# Section 3: Sélection et jointure de données

In [ ]:
import sqlite3

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/persons_cleaned.csv')
df

In [ ]:
haircuts = pd.read_csv('https://raw.githubusercontent.com/tintamarre/python-introduction/main/datasets/persons_haircut.csv')
haircuts

In [ ]:
# quick cleaning 
# haircuts = haircuts.dropna()

# Replace mULLET by Mullet in ['haircut']
haircuts['haircut'] = haircuts['haircut'].str.replace('mULLET', 'Mullet')

# rename column NATIONAL_NUMBER_CAPS_LOVER to national_number
haircuts = haircuts.rename(columns={'NATIONAL_NUMBER_CAPS_LOVER': 'national_number'})

In [ ]:
df = df.merge(haircuts, on='national_number', how='left')

In [ ]:
# remove non numeric values from the column national_number
df['national_number'] = df['national_number'].str.replace(r'\D', '', regex=True)

df = df.merge(haircuts, on='national_number', how='left')


In [ ]:
haircuts.dtypes

In [ ]:
#df['national_number'] = df['national_number'].str.replace(r'\D', '', regex=True)

# https://ihateregex.io/

df['national_number'] = df['national_number'].astype(int)

df_merge = df.merge(haircuts, on='national_number', how='left')
df_merge

## SQL

In [ ]:
# https://www.sqlite.org/
con = sqlite3.connect('./datasets/persons.db')

df.to_sql('persons', con, if_exists='replace', index=False)
haircuts.to_sql('haircuts', con, if_exists='replace', index=False)

In [ ]:
# drop first_name, last_name, national_number
df_merge = df_merge.drop(columns=['national_number'])

In [ ]:
df_merge.sample(10)

In [ ]:
# haircuts nan -> 'Bald'
df_merge['haircut'] = df_merge['haircut'].fillna('Bald')

# replace k in white_hair_count with 1000
df_merge['white_hair_count'] = df_merge['white_hair_count'].str.replace('k', '000')
df_merge['white_hair_count'] = df_merge['white_hair_count'].fillna(0)
df_merge['white_hair_count'] = df_merge['white_hair_count'].astype(int)

In [ ]:
df_merge

In [ ]:
import plotly.express as px

# plot sum of persons with by haircut
fig = px.bar(df_merge.groupby('haircut').size().reset_index(name='count'), x='haircut', y='count')
fig.show()

In [ ]:
# remove Bald persons
df_merge = df_merge[df_merge['haircut'] != 'Bald']

# plot sum of persons with by haircut
fig = px.bar(df_merge.groupby('haircut').size())
fig.show()

In [ ]:
import plotly.express as px

# plot sum of white_hair_count by haircut
fig = px.bar(
        df_merge.groupby('haircut').agg({'white_hair_count': 'mean'}).reset_index(), 
        x='haircut', 
        y='white_hair_count')
fig.show()


In [ ]:
# https://www.sqlite.org/
con = sqlite3.connect('./datasets/persons.db')

df_from_sql = pd.read_sql('SELECT * FROM persons LEFT JOIN haircuts ON persons.national_number = haircuts.national_number', con)
df_from_sql